# Projeto MegaDados

**Alunos:**
    
+ Rafael Almada
+ Warlen Cesar Rodrigues
    
**Professor:** Fábio Ayres

In [130]:
import pyspark
from math import log10

In [131]:
sc = pyspark.SparkContext(appName='Projeto e-sports')

In [132]:
sc

<SparkContext master=local[*] appName=Projeto e-sports>

In [133]:
rdd = sc.sequenceFile('part-00000')
rdd

MapPartitionsRDD[2] at mapPartitions at SerDeUtil.scala:244

In [134]:
# result = rdd.take(10)
# result[0]

### Montando RDD para Valorant

In [135]:
#Pegar os documentos que possuem a palavra Valorant
def getValorantWord(item):
    if 'Valorant' in item[1]:
        return item
    
rdd_valorant = rdd.filter(getValorantWord)
rdd_valorant.count()

10

### Montando RDD para Counter Strike

In [136]:
#Pegar os documentos que possuem a palavra Counter Strike
def getCStWord(item):
    if 'Counter Strike' in item[1]:
        return item
    
rdd_csgo = rdd.filter(getCStWord)
rdd_csgo.count()

14

### Operacionalização para Intersecção entre Valorant e Counter Strike

In [137]:
#Pegar somente os documentos que possuem as duas palavras 
rdd_valorant_csgo = rdd_valorant.intersection(rdd_csgo)
rdd_valorant_csgo.count()

1

#### Operacionalizando frequência de palavras (1 doc = 1 contagem)

In [138]:
def conta_palavras(item):
    url, conteudo = item
    palavras = conteudo.strip().split()
    return [(palavra, 1) for palavra in set(palavras)]

def junta_contagens(nova_contagem, contagem_atual):
    return nova_contagem + contagem_atual

rdd_valorant_csgo_doc_freq = rdd_valorant_csgo.flatMap(conta_palavras).reduceByKey(junta_contagens)

In [139]:
#Listando tuplas (palavra, qtde. de Docs) para intersecção de Valorant e CSGO
#Neste cenário, o valor máximo do value é a quantidade retornada por rdd_valorant_csgo.count()
result = rdd_valorant_csgo_doc_freq.collect() 
sorted(result, key=lambda x : -x[1])

[('habitualmente,', 1),
 ('Blade', 1),
 ('assim', 1),
 ('no', 1),
 ('Como', 1),
 ('Legends', 1),
 ('RK', 1),
 ('frequentes', 1),
 ('Assinaturas', 1),
 ('10', 1),
 ('direitos', 1),
 ('Serviços', 1),
 ('Combos', 1),
 ('podem', 1),
 ('Mouses', 1),
 ('respondido.', 1),
 ('Awakening', 1),
 ('of', 1),
 ('perfil', 1),
 ('donos.', 1),
 ('tíquete', 1),
 ('BRL', 1),
 ('recente', 1),
 ('Fortnite', 1),
 ('mesmo', 1),
 ('Suporte', 1),
 ('Os', 1),
 ('Troca', 1),
 ('dentro', 1),
 ('automatização', 1),
 ('lista', 1),
 ('Licenças', 1),
 ('logotipos', 1),
 ('Montarias', 1),
 ('Alz', 1),
 ('Fechar', 1),
 ('Teamfight', 1),
 ('Fire', 1),
 ('Grand', 1),
 ('ser', 1),
 ('categorias', 1),
 ('(email', 1),
 ('sendo', 1),
 ('Perguntas', 1),
 ('conta', 1),
 ('DFG', 1),
 ('filipecobr', 1),
 ('propriedade', 1),
 ('R$250', 1),
 ('uso', 1),
 ('necessário.', 1),
 ('Jogos', 1),
 ('and', 1),
 ('Política', 1),
 ('UP', 1),
 ('espera.', 1),
 ('War', 1),
 ('215', 1),
 ('(USD)', 1),
 ('34', 1),
 ('Steam', 1),
 ('Gift', 1),
 (

#### Fazendo o recorte de palavras muito específicas ou muito comuns

In [140]:
DOC_COUNT_MIN = 2 # Isso deve mudar de amostra para amostra
DOC_COUNT_MAX = 0.7 * rdd.count() # Isso deve mudar de amostra para amostra (0.7 * rdd.count())

def filtra_doc_freq(item):
    contagem = item[1]
    return (contagem < DOC_COUNT_MAX) & (contagem >= DOC_COUNT_MIN)

rdd_valorant_csgo_freq_filtrado = rdd_valorant_csgo_doc_freq.filter(filtra_doc_freq)

In [141]:
rdd_valorant_csgo_freq_filtrado.take(10)

[]

#### Calculo do IDF para as palavras encontradas em documentos que possuem Valorant E Counter Strike

In [142]:
def idf(item):
    palavra, contagem = item
    return (palavra, log10(N/contagem))

rdd_idf = rdd_valorant_csgo_freq_filtrado.map(idf)
result = rdd_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

[]

#### Calculo da frequencia e relevância das palavras

In [143]:
def freq_palavra(item):
    palavra, contagem = item
    return (palavra, log10(1+contagem))

def relevancia(item):
    palavra, freq = item
    freq_calc = freq_palavra(item)
    idf_calc = idf(item)
    relev = freq_calc[1]*idf_calc[1]
    return (palavra, relev)

In [144]:
freq_filtrado_sort_by_key_rdd = rdd_valorant_csgo_freq_filtrado.sortByKey()
rdd_relevancia_ambas_palavras_mesmo_doc = freq_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_ambas_palavras_mesmo_doc.collect()
sorted(res, key=lambda x : x[1])[-100:]

[]

### Operacionalização para exclusão mútua entre Valorant e Counter Strike
### Counter Strike

#### Criando o RDD com documentos que possuem somente Counter Strike
#### Filtrando o RDD

In [145]:
rdd_csgo_only = rdd_csgo.subtract(rdd_valorant_csgo)
rdd_csgo_only_doc_freq = rdd_csgo_only.flatMap(conta_palavras).reduceByKey(junta_contagens)
rdd_csgo_only_doc_freq_filtrado = rdd_csgo_only_doc_freq.filter(filtra_doc_freq)
rdd_csgo_only_doc_freq_filtrado.take(10)

[('Joomla', 5),
 ('no', 13),
 ('Life', 6),
 ('fulls', 5),
 ('Bate-papo', 5),
 ('links', 5),
 ('Tales', 5),
 ('+]', 5),
 ('of', 12),
 ('Dúvidas]', 5)]

#### Calculando o IDF das palavras

In [146]:
rdd_csgo_only_idf = rdd_csgo_only_doc_freq_filtrado.map(idf)
result = rdd_csgo_only_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

[('vou', 4.256874025393585),
 ('25', 4.256874025393585),
 ('Jogo', 4.256874025393585),
 ('papel', 4.256874025393585),
 ('Próximo', 4.256874025393585),
 ('marcados', 4.256874025393585),
 ('Login', 4.256874025393585),
 ('Nada', 4.256874025393585),
 ('Tema', 4.256874025393585),
 ('Vender', 4.256874025393585),
 ('Ultimate', 4.256874025393585),
 ('melhores', 4.256874025393585),
 ('fps', 4.256874025393585),
 ('Ram', 4.256874025393585),
 ('Jogar?', 4.256874025393585),
 ('TELEFONE:', 4.256874025393585),
 ('condições', 4.256874025393585),
 ("Playerunknown's", 4.256874025393585),
 ('#1', 4.256874025393585),
 ('isso', 4.256874025393585),
 ('33', 4.256874025393585),
 ('personalizados', 4.256874025393585),
 ('JavaScript', 4.256874025393585),
 ('Erro', 4.256874025393585),
 ('HOME', 4.256874025393585),
 ('comentar.', 4.256874025393585),
 ('R$', 4.256874025393585),
 ('livro', 4.256874025393585),
 ('Mouse', 4.256874025393585),
 ('Cooler', 4.256874025393585),
 ('Sexta', 4.256874025393585),
 ('Placa', 4.

#### Calculando a relevância das palavras

In [147]:
csgo_filtrado_sort_by_key_rdd = rdd_csgo_only_doc_freq_filtrado.sortByKey()
rdd_relevancia_csgo_only = csgo_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_csgo_only.collect()
sorted(res, key=lambda x : x[1])[-100:]

[('is', 3.1942616582349888),
 ('mensagens', 3.1942616582349888),
 ('neste', 3.1942616582349888),
 ('outros', 3.1942616582349888),
 ('pedidos', 3.1942616582349888),
 ('reserved.', 3.1942616582349888),
 ('rights', 3.1942616582349888),
 ('servidor', 3.1942616582349888),
 ('servidores', 3.1942616582349888),
 ('suas', 3.1942616582349888),
 ('tempo', 3.1942616582349888),
 ('todos', 3.1942616582349888),
 ('vBulletin', 3.1942616582349888),
 ('&', 3.352997905124013),
 ('Brasil', 3.352997905124013),
 ('Como', 3.352997905124013),
 ('Fórum', 3.352997905124013),
 ('Imagens', 3.352997905124013),
 ('Nintendo', 3.352997905124013),
 ('Notícias', 3.352997905124013),
 ('Outros', 3.352997905124013),
 ('Para', 3.352997905124013),
 ('Playstation', 3.352997905124013),
 ('Quem', 3.352997905124013),
 ('Seja', 3.352997905124013),
 ('Team', 3.352997905124013),
 ('Vídeos', 3.352997905124013),
 ('Web', 3.352997905124013),
 ('X', 3.352997905124013),
 ('aqui', 3.352997905124013),
 ('conteúdo', 3.352997905124013),
 (

### Valorant

#### Criando o RDD com documentos que possuem somente Counter Strike
#### Filtrando o RDD

In [148]:
rdd_valorant_only = rdd_valorant.subtract(rdd_valorant_csgo)
rdd_valorant_only_doc_freq = rdd_valorant_only.flatMap(conta_palavras).reduceByKey(junta_contagens)
rdd_valorant_only_doc_freq_filtrado = rdd_valorant_only_doc_freq.filter(filtra_doc_freq)
rdd_valorant_only_doc_freq_filtrado.take(10)

[('Among', 2),
 ('boa', 2),
 ('baseado', 2),
 ('das', 4),
 ('no', 7),
 ('Só', 2),
 ('favorito', 2),
 ('podem', 3),
 ('of', 6),
 ('melhor', 3)]

#### Calculando o IDF das palavras

In [149]:
rdd_valorant_only_idf = rdd_valorant_only_doc_freq_filtrado.map(idf)
result = rdd_valorant_only_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

[('semifinais', 4.256874025393585),
 ('eSports', 4.256874025393585),
 ('relativamente', 4.256874025393585),
 ('Colunistas', 4.256874025393585),
 ('esta', 4.256874025393585),
 ('one', 4.256874025393585),
 ('viole', 4.256874025393585),
 ('Thiago', 4.256874025393585),
 ('preços', 4.256874025393585),
 ('gosta', 4.256874025393585),
 ('eles', 4.256874025393585),
 ('confira', 4.256874025393585),
 ('séries', 4.256874025393585),
 ('Desfazer', 4.256874025393585),
 ('R$0,00', 4.256874025393585),
 ('Item', 4.256874025393585),
 ('Talvez', 4.256874025393585),
 ('Ragnarok', 4.256874025393585),
 ('Star', 4.256874025393585),
 ('adiado', 4.256874025393585),
 ('globo.com', 4.256874025393585),
 ('g7', 4.256874025393585),
 ('site.', 4.256874025393585),
 ('Claro,', 4.256874025393585),
 ('ilegal.', 4.256874025393585),
 ('Melhores', 4.256874025393585),
 ('Monitor', 4.256874025393585),
 ('anos,', 4.256874025393585),
 ('ano', 4.256874025393585),
 ('19', 4.256874025393585),
 ('porque', 4.256874025393585),
 ('con

#### Calculando a relevância das palavras

In [150]:
valorant_filtrado_sort_by_key_rdd = rdd_valorant_only_doc_freq_filtrado.sortByKey()
rdd_relevancia_valorant_only = valorant_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_valorant_only.collect()
sorted(res, key=lambda x : x[1])[-100:]

[('World', 2.7650163186127155),
 ('algumas', 2.7650163186127155),
 ('anuncia', 2.7650163186127155),
 ('ao', 2.7650163186127155),
 ('cada', 2.7650163186127155),
 ('casa', 2.7650163186127155),
 ('chega', 2.7650163186127155),
 ('conta', 2.7650163186127155),
 ('das', 2.7650163186127155),
 ('disponíveis', 2.7650163186127155),
 ('final', 2.7650163186127155),
 ('games', 2.7650163186127155),
 ('hoje', 2.7650163186127155),
 ('ideia', 2.7650163186127155),
 ('jogos', 2.7650163186127155),
 ('maior', 2.7650163186127155),
 ('mas', 2.7650163186127155),
 ('muito', 2.7650163186127155),
 ('nos', 2.7650163186127155),
 ('nossa', 2.7650163186127155),
 ('nova', 2.7650163186127155),
 ('novo', 2.7650163186127155),
 ('oficial', 2.7650163186127155),
 ('opinião', 2.7650163186127155),
 ('pela', 2.7650163186127155),
 ('primeiro', 2.7650163186127155),
 ('será', 2.7650163186127155),
 ('são', 2.7650163186127155),
 ('também', 2.7650163186127155),
 ('tempo', 2.7650163186127155),
 ('the', 2.7650163186127155),
 ('todos',

In [129]:
sc.stop()